In [2]:

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import torchvision
from torchsummary import summary
from torchvision import transforms
import sys

sys.path.append("/workspaces/ood/")
from scripts_ood.train import train_linear_model
import warnings
import json
from PIL import Image
import torch
from torchvision import transforms
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("/workspaces/ood/")
from scripts_ood.utils import imagenet_sanity_check
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import timm

from scripts_ood.utils import load_pretrained_weights_vit
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2


device = torch.device("cuda")
torch.cuda.is_available()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [3]:
# model_name = 'byol'
# in_dist_data = 'svhn'
# linear_model = train_linear_model(model_name=model_name, device=device, emb_size=2048, in_dist_data=in_dist_data, train_epoch=100, init_lr=1e-4)
# torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_{in_dist_data}_linear_model.pth')

### 1) Load data

In [4]:
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# transform = transforms.Compose(
#     [
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         normalize,
#     ]
# )

# vit_base_patch16_384 no linear image net head

In [5]:
model_name = 'vit_base_patch16_384'
model = timm.create_model(model_name, pretrained=True, num_classes=0).to(device)
model.eval();
model_name = 'vit_base_patch16_384_nohead'

In [6]:
config = resolve_data_config({}, model=model)
transform = create_transform(**config)
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)
cifar_data_test = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform, train=False
)

svhn_data_train = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform
)
svhn_data_test = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform, split="test"
)

In [7]:
from scripts_ood.utils import predict_on_whole_dataset


predict_on_whole_dataset(model, svhn_data_test, f"{model_name}_svhn_test", device)
predict_on_whole_dataset(model, cifar_data_test, f"{model_name}_cifar_test", device)
predict_on_whole_dataset(model, cifar_data_train, f"{model_name}_cifar_train", device)
predict_on_whole_dataset(model, svhn_data_train, f"{model_name}_svhn_train", device)


Predictions are already present in /workspaces/ood/data/predictions/vit_base_patch16_384_nohead_svhn_test.npy
Predictions are already present in /workspaces/ood/data/predictions/vit_base_patch16_384_nohead_cifar_test.npy
Predictions are already present in /workspaces/ood/data/predictions/vit_base_patch16_384_nohead_cifar_train.npy
Predictions are already present in /workspaces/ood/data/predictions/vit_base_patch16_384_nohead_svhn_train.npy


In [8]:
#linear_model = train_linear_model(model_name=model_name, device=device, emb_size=768)
in_dist_data = 'svhn'
linear_model = train_linear_model(model_name=model_name, device=device, emb_size=768, in_dist_data=in_dist_data)

epoch: 0, train loss: 2.56401,  train_accuracy: 0.11248, test_accuracy 0.11056
current LR: 0.0001
epoch: 1, train loss: 1.71622,  train_accuracy: 0.41539, test_accuracy 0.42340
current LR: 0.0001
epoch: 2, train loss: 1.58441,  train_accuracy: 0.48563, test_accuracy 0.49985
current LR: 0.0001
epoch: 3, train loss: 1.51112,  train_accuracy: 0.51928, test_accuracy 0.53519
current LR: 0.0001
epoch: 4, train loss: 1.41367,  train_accuracy: 0.54152, test_accuracy 0.56054
current LR: 0.0001
epoch: 5, train loss: 1.38661,  train_accuracy: 0.55537, test_accuracy 0.57137
current LR: 0.0001
epoch: 6, train loss: 1.46041,  train_accuracy: 0.56588, test_accuracy 0.58313
current LR: 0.0001
epoch: 7, train loss: 1.35759,  train_accuracy: 0.57414, test_accuracy 0.59008
current LR: 0.0001
epoch: 8, train loss: 1.36534,  train_accuracy: 0.58228, test_accuracy 0.59577
current LR: 0.0001
epoch: 9, train loss: 1.34415,  train_accuracy: 0.58400, test_accuracy 0.60134
current LR: 0.0001
epoch: 10, train los

In [ ]:
torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_{in_dist_data}_linear_model.pth')

# vit_base_patch16_384 with 1000 head

In [ ]:

model_name = 'vit_base_patch16_384'
model = timm.create_model(model_name, pretrained=True, ).to(device)
model.eval();

In [ ]:

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

imagenet_sanity_check(model, transform, device)

In [ ]:
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)
cifar_data_test = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform, train=False
)

svhn_data_train = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform
)
svhn_data_test = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform, split="test"
)

In [ ]:
from scripts_ood.utils import add_labels, predict_on_whole_dataset

predict_on_whole_dataset(model, svhn_data_test, f"{model_name}_svhn_test", device)
predict_on_whole_dataset(model, cifar_data_test, f"{model_name}_cifar_test", device)
predict_on_whole_dataset(model, cifar_data_train, f"{model_name}_cifar_train", device)

In [ ]:
linear_model = train_linear_model(model_name=model_name, device=device, emb_size=1000)

In [ ]:
torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_cifar_linear_model.pth')

# Softmax ImageNet resnet50

In [ ]:
soft_model = torchvision.models.resnet50(pretrained=False).to(device)
soft_model.load_state_dict(
    torch.load(
        "/workspaces/ood/data/models/torch/hub/checkpoints/resnet50-0676ba61.pth"
    )
)
summary(soft_model, input_size=(3, 224, 224))

In [ ]:
from ood.utils import imagenet_sanity_check

imagenet_sanity_check(soft_model, transform, device)

In [ ]:
from scripts_ood.utils import add_labels, predict_on_whole_dataset

predict_on_whole_dataset(soft_model, cifar_data_train, "soft_cifar_train")

In [ ]:
model_name = "soft"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=1000)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/soft_cifar_linear_model0.88test_accuracy.pth')

# MoCo v2 ImageNet pretrain

In [ ]:
from ood.utils import load_moco

model = load_moco("/workspaces/ood/data/models/moco_v2_800ep_pretrain.pth.tar", device)

In [ ]:
from ood.utils import add_labels, predict_on_whole_dataset_moco

predict_on_whole_dataset_moco(model, cifar_data_test, "moco_cifar_test", device)
predict_on_whole_dataset_moco(model, cifar_data_train, "moco_cifar_train", device)
predict_on_whole_dataset_moco(model, svhn_data_train, "moco_svhn_train", device)
predict_on_whole_dataset_moco(model, svhn_data_test, "moco_svhn_test", device)

In [ ]:
model_name = "moco"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=2048)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/moco_cifar_linear_model0.8382test_accuracy.pth')

# BYOL ImageNet pretrain

In [ ]:
from ood.utils import load_byol

model = load_byol("/workspaces/ood/data/models/pretrain_res50x1.pth.tar", device)

model_name = "byol"
predict_on_whole_dataset(model, cifar_data_test, f"{model_name}_cifar_test", device)
predict_on_whole_dataset(model, cifar_data_train, f"{model_name}_cifar_train", device)
predict_on_whole_dataset(model, svhn_data_train, f"{model_name}_svhn_train", device)
predict_on_whole_dataset(model, svhn_data_test, f"{model_name}_svhn_test", device)

In [ ]:
model_name = "byol"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=2048)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_cifar_linear_model0.905test_accuracy.pth')